<a href="https://colab.research.google.com/github/ziadelsharawy/Test/blob/main/Customer_Churn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install xgboost


In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from scipy.stats import f_oneway
from scipy.stats import chi2_contingency
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
import xgboost as xgb
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import GridSearchCV


In [3]:
df=pd.read_csv(r"/content/WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [5]:
df.describe()

,SeniorCitizen,tenure,MonthlyCharges
count,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692
std,0.368612,24.559481,30.090047
min,0.000000,0.000000,18.250000
25%,0.000000,9.000000,35.500000
50%,0.000000,29.000000,70.350000
75%,0.000000,55.000000,89.850000
max,1.000000,72.000000,118.750000


In [6]:
df.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [7]:
s = (df.dtypes == 'object')
object_cols = list(s[s].index)
for i in object_cols:
    print(df[i].value_counts())

customerID
7590-VHVEG    1
3791-LGQCY    1
6008-NAIXK    1
5956-YHHRX    1
5365-LLFYV    1
             ..
9796-MVYXX    1
2637-FKFSY    1
1552-AAGRX    1
4304-TSPVK    1
3186-AJIEK    1
Name: count, Length: 7043, dtype: int64
gender
Male      3555
Female    3488
Name: count, dtype: int64
Partner
No     3641
Yes    3402
Name: count, dtype: int64
Dependents
No     4933
Yes    2110
Name: count, dtype: int64
PhoneService
Yes    6361
No      682
Name: count, dtype: int64
MultipleLines
No                  3390
Yes                 2971
No phone service     682
Name: count, dtype: int64
InternetService
Fiber optic    3096
DSL            2421
No             1526
Name: count, dtype: int64
OnlineSecurity
No                     3498
Yes                    2019
No internet service    1526
Name: count, dtype: int64
OnlineBackup
No                     3088
Yes                    2429
No internet service    1526
Name: count, dtype: int64
DeviceProtection
No                     3095
Yes               

In [8]:
df.duplicated().sum()

0

In [9]:
df.shape

(7043, 21)

In [10]:
label_encoder = LabelEncoder()
df["Churn"] = label_encoder.fit_transform(df["Churn"])
df["Churn"].unique()

array([0, 1])

In [11]:
data_num=["tenure","MonthlyCharges","TotalCharges"]
data_cat=['customerID','gender','Partner','Dependents','PhoneService','MultipleLines','InternetService','OnlineSecurity','OnlineBackup','DeviceProtection',
          'TechSupport','StreamingTV','StreamingMovies','Contract','PaperlessBilling','PaymentMethod',"SeniorCitizen"]

In [12]:
for feature in data_num:
    categories = df[feature].unique()
    samples = [df[df[feature] == category]['Churn'] for category in categories]
    f_statistic, p_value = f_oneway(*samples)

    print(f"ANOVA for {feature}:")
    print("F-statistic:", f_statistic)
    print("P-value:", p_value)
    print()

    # Interpretation of the p-value
    alpha = 0.05  # Significance level
    if p_value < alpha:
        print("Reject the null hypothesis - There is a significant relationship between the variables.")
    else:
        print("Fail to reject the null hypothesis - There is no significant relationship between the variables.")


ANOVA for tenure:
F-statistic: 17.252534887236887
P-value: 1.3637087667530093e-193

Reject the null hypothesis - There is a significant relationship between the variables.
ANOVA for MonthlyCharges:
F-statistic: 1.4874473665576302
P-value: 1.0992885081459628e-24

Reject the null hypothesis - There is a significant relationship between the variables.
ANOVA for TotalCharges:
F-statistic: 0.9655867649131552
P-value: 0.7124861680681824

Fail to reject the null hypothesis - There is no significant relationship between the variables.


In [13]:
drop_column=[]
for feature in data_cat:
    # Create a contingency table
    contingency_table = pd.crosstab(df[feature], df['Churn'])

    # Perform the chi-square test
    chi2, p, dof, expected = chi2_contingency(contingency_table)

    # Print the results
    print(f"Chi-square Test for {feature}:")
    print("Chi-square statistic:", chi2)
    print("P-value:", p)
    print("Degrees of freedom:", dof)
    print("Expected frequencies:")
    print(expected)
    print()

    alpha = 0.05  # Significance level
    if p < alpha:
        print("Reject the null hypothesis - There is a significant relationship between the variables.")
    else:
      drop_column.append(feature)
      print("Fail to reject the null hypothesis - There is no significant relationship between the variables.")

Chi-square Test for customerID:
Chi-square statistic: 7043.0
P-value: 0.49439767459438705
Degrees of freedom: 7042
Expected frequencies:
[[0.73463013 0.26536987]
 [0.73463013 0.26536987]
 [0.73463013 0.26536987]
 ...
 [0.73463013 0.26536987]
 [0.73463013 0.26536987]
 [0.73463013 0.26536987]]

Fail to reject the null hypothesis - There is no significant relationship between the variables.
Chi-square Test for gender:
Chi-square statistic: 0.4840828822091383
P-value: 0.48657873605618596
Degrees of freedom: 1
Expected frequencies:
[[2562.38989067  925.61010933]
 [2611.61010933  943.38989067]]

Fail to reject the null hypothesis - There is no significant relationship between the variables.
Chi-square Test for Partner:
Chi-square statistic: 158.7333820309922
P-value: 2.1399113440759935e-36
Degrees of freedom: 1
Expected frequencies:
[[2674.78830044  966.21169956]
 [2499.21169956  902.78830044]]

Reject the null hypothesis - There is a significant relationship between the variables.
Chi-squar

In [14]:
drop_column

['customerID', 'gender', 'PhoneService']

In [15]:
df.drop('customerID' ,axis=1,inplace=True)
df.drop('gender' ,axis=1,inplace=True)
df.drop('PhoneService' ,axis=1,inplace=True)
df.drop('TotalCharges' ,axis=1,inplace=True)
df.columns

Index(['SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'MultipleLines',
       'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
       'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges', 'Churn'],
      dtype='object')

In [16]:
categorical_data=['SeniorCitizen', 'Partner', 'Dependents', 'MultipleLines',
       'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
       'PaperlessBilling', 'PaymentMethod']

In [17]:
#Initialize OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)

# Apply one-hot encoding to the categorical columns
one_hot_encoded = encoder.fit_transform(df[categorical_data])

#Create a DataFrame with the one-hot encoded columns
#We use get_feature_names_out() to get the column names for the encoded data
one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(categorical_data))

# Concatenate the one-hot encoded dataframe with the original dataframe
df_encoded = pd.concat([df, one_hot_df], axis=1)

# Drop the original categorical columns
df_encoded = df_encoded.drop(categorical_data, axis=1)
df_encoded.head()

,tenure,MonthlyCharges,Churn,SeniorCitizen_0,SeniorCitizen_1,Partner_No,Partner_Yes,Dependents_No,Dependents_Yes,MultipleLines_No,...,StreamingMovies_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,PaperlessBilling_No,PaperlessBilling_Yes,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,1,29.85,0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,34,56.95,0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,2,53.85,1,1.0,0.0,1.0,0.0,1.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,45,42.30,0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
4,2,70.70,1,1.0,0.0,1.0,0.0,1.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [18]:
df_encoded.columns

Index(['tenure', 'MonthlyCharges', 'Churn', 'SeniorCitizen_0',
       'SeniorCitizen_1', 'Partner_No', 'Partner_Yes', 'Dependents_No',
       'Dependents_Yes', 'MultipleLines_No', 'MultipleLines_No phone service',
       'MultipleLines_Yes', 'InternetService_DSL',
       'InternetService_Fiber optic', 'InternetService_No',
       'OnlineSecurity_No', 'OnlineSecurity_No internet service',
       'OnlineSecurity_Yes', 'OnlineBackup_No',
       'OnlineBackup_No internet service', 'OnlineBackup_Yes',
       'DeviceProtection_No', 'DeviceProtection_No internet service',
       'DeviceProtection_Yes', 'TechSupport_No',
       'TechSupport_No internet service', 'TechSupport_Yes', 'StreamingTV_No',
       'StreamingTV_No internet service', 'StreamingTV_Yes',
       'StreamingMovies_No', 'StreamingMovies_No internet service',
       'StreamingMovies_Yes', 'Contract_Month-to-month', 'Contract_One year',
       'Contract_Two year', 'PaperlessBilling_No', 'PaperlessBilling_Yes',
       'PaymentMet

In [19]:
X=df_encoded.drop(columns=["Churn"])
y=df_encoded["Churn"]

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
poly = PolynomialFeatures(degree=2, include_bias=False)  # Experiment with degree
X_train = poly.fit_transform(X_train)
X_test = poly.transform(X_test)

In [24]:
models = {
    "Logistic Regression": LogisticRegression(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "Support Vector Machine": SVC(),
    "K-Nearest Neighbors": KNeighborsClassifier(),
    "Gaussian Naive Bayes": GaussianNB(),
    "Gradient Boosting":GradientBoostingClassifier(),
    "XGBoost": xgb.XGBClassifier(),
}
# Define the parameter grids for each model
param_grids = {
    "XGBoost": {
        'xgbclassifier__n_estimators': [50, 100, 200],
        'xgbclassifier__max_depth': [3, 6, 9],
        'xgbclassifier__learning_rate': [0.01, 0.1, 0.2],
        'xgbclassifier__subsample': [0.8, 1.0],
        'xgbclassifier__colsample_bytree': [0.8, 1.0]
    },
    "Random Forest": {
        'randomforestclassifier__n_estimators': [200,250,300],
        'randomforestclassifier__max_depth': [None, 5,10,15,20],
        'randomforestclassifier__min_samples_split': [10,15,20],
        'randomforestclassifier__min_samples_leaf': [4,5,6,7]
    },
    "Decision Tree": {
        'decisiontreeclassifier__max_depth': [None,5, 10, 20, 30],
        'decisiontreeclassifier__min_samples_split': [2, 5, 10],
        'decisiontreeclassifier__min_samples_leaf': [5,10,15]

    },
    "K-Nearest Neighbors": {
        'kneighborsclassifier__n_neighbors': [3, 5, 7, 9],
        'kneighborsclassifier__weights': ['uniform', 'distance'],
        'kneighborsclassifier__metric': ['euclidean', 'manhattan', 'minkowski']
    },
    "Gaussian Naive Bayes": {
        'gaussiannb__var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5]

    }
}

# Apply models and classification reports with hyperparameter tuning
for name, model in models.items():
    pipeline = make_pipeline(StandardScaler(), model)

    if name in param_grids:
        grid_search = GridSearchCV(pipeline, param_grids[name], cv=5, scoring='accuracy')
        grid_search.fit(X_train, y_train)
        best_model = grid_search.best_estimator_
        print(f"Best parameters for {name}: {grid_search.best_params_}")
        print(f"Best cross-validation accuracy for {name}: {grid_search.best_score_}")
    else:
        best_model = pipeline.fit(X_train, y_train)

    y_pred_test = best_model.predict(X_test)
    y_pred_train = best_model.predict(X_train)

    accuracy_test = accuracy_score(y_test, y_pred_test)
    accuracy_train = accuracy_score(y_train, y_pred_train)

    print(f"{name}: Accuracy-test = {accuracy_test:.2f}\n")
    print(f"{name}: Accuracy-train = {accuracy_train:.2f}\n")

    print(classification_report(y_test, y_pred_test))
    print(classification_report(y_train, y_pred_train))

    print("="*50)  # Separating lines for better readability


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression: Accuracy-test = 0.81

Logistic Regression: Accuracy-train = 0.81

              precision    recall  f1-score   support

           0       0.85      0.90      0.87      1036
           1       0.67      0.55      0.60       373

    accuracy                           0.81      1409
   macro avg       0.76      0.72      0.74      1409
weighted avg       0.80      0.81      0.80      1409

              precision    recall  f1-score   support

           0       0.85      0.90      0.88      4138
           1       0.68      0.55      0.61      1496

    accuracy                           0.81      5634
   macro avg       0.76      0.73      0.74      5634
weighted avg       0.80      0.81      0.80      5634

Best parameters for Decision Tree: {'decisiontreeclassifier__max_depth': 5, 'decisiontreeclassifier__min_samples_leaf': 10, 'decisiontreeclassifier__min_samples_split': 5}
Best cross-validation accuracy for Decision Tree: 0.7889584098370215
Decision Tree: Acc

In [25]:
modeles = {
     "Random Forest": RandomForestClassifier()
}
# Define the parameter grids for each model
param_grids = {

    "Random Forest": {
        'randomforestclassifier__n_estimators': [150, 200.250],
        'randomforestclassifier__max_depth': [None, 5,10,15, 20, 30],
        'randomforestclassifier__min_samples_split': [2, 5, 10,15],
        'randomforestclassifier__min_samples_leaf': [1, 2,4,5]
    }
}

# Apply models and classification reports with hyperparameter tuning
for name, model in modeles.items():
    pipeline = make_pipeline(StandardScaler(), model)

    if name in param_grids:
        grid_search = GridSearchCV(pipeline, param_grids[name], cv=5, scoring='accuracy')
        grid_search.fit(X_train, y_train)
        best_model = grid_search.best_estimator_
        print(f"Best parameters for {name}: {grid_search.best_params_}")
        print(f"Best cross-validation accuracy for {name}: {grid_search.best_score_}")
    else:
        best_model = pipeline.fit(X_train, y_train)

    y_pred_test = best_model.predict(X_test)
    y_pred_train = best_model.predict(X_train)

    accuracy_test = accuracy_score(y_test, y_pred_test)
    accuracy_train = accuracy_score(y_train, y_pred_train)

    print(f"{name}: Accuracy-test = {accuracy_test:.2f}\n")
    print(f"{name}: Accuracy-train = {accuracy_train:.2f}\n")

    print(classification_report(y_test, y_pred_test))
    print(classification_report(y_train, y_pred_train))

    print("="*50)  # Separating lines for better readability


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
480 fits failed out of a total of 960.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
480 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 405, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py", line 340, in fit
    self._validate_params()
  File "/usr/local/lib/python3.10/dist-pac

Best parameters for Random Forest: {'randomforestclassifier__max_depth': 10, 'randomforestclassifier__min_samples_leaf': 5, 'randomforestclassifier__min_samples_split': 15, 'randomforestclassifier__n_estimators': 150}
Best cross-validation accuracy for Random Forest: 0.7999634358338286
Random Forest: Accuracy-test = 0.81

Random Forest: Accuracy-train = 0.85

              precision    recall  f1-score   support

           0       0.85      0.91      0.88      1036
           1       0.68      0.54      0.60       373

    accuracy                           0.81      1409
   macro avg       0.76      0.72      0.74      1409
weighted avg       0.80      0.81      0.80      1409

              precision    recall  f1-score   support

           0       0.87      0.94      0.90      4138
           1       0.77      0.61      0.68      1496

    accuracy                           0.85      5634
   macro avg       0.82      0.77      0.79      5634
weighted avg       0.84      0.85      